In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Keypress

In [36]:
#keyPress = pd.read_csv('../agg_datasets/KeyPressEvent.csv',index_col = 0)
keyPress = pd.read_csv('./../Our Data/KeyPressEvent.csv',index_col = 0)

In [37]:
#keyPress = keyPress.rename(columns={'PressType':'ActivityID','ActivityID':'PressType'})

In [38]:
#keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6).astype(int)
keyPress['SessionID'] = keyPress.ActivityID

In [39]:
aux = keyPress[['Systime','SessionID']].groupby('SessionID').agg([np.min,np.max]).Systime

In [40]:
aux['diff'] = aux.amax - aux.amin
aux = aux.reset_index()

In [41]:
bla = []
step = 10000
for _,row in aux.iterrows():
    i = 0
    t0 = row.amin
    while (t0+step < row.amax):
        aux_dict = {}
        aux_dict['SessionID'] = row.SessionID
        aux_dict['WindowNumber'] = i
        aux_dict['WindowStart'] = t0
        aux_dict['WindowEnd'] = t0+step
        t0 += step
        i += 1
        bla.append(aux_dict)

In [42]:
fim = pd.DataFrame(bla)

In [43]:
fim.to_csv('skeleton.csv',index=False)

In [44]:
skeleton = pd.read_csv('skeleton.csv')

In [45]:
%%time
aux = keyPress.merge(fim,on='SessionID')

CPU times: user 462 ms, sys: 276 ms, total: 738 ms
Wall time: 791 ms


In [46]:
aux = aux[(aux.Systime >= aux.WindowStart) & (aux.Systime <= aux.WindowEnd)]

In [47]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

In [48]:
keyPress.to_csv('../Our Data Windowed/KeyPressEvent.csv',index=False)

# Touch Event

In [49]:
#keyPress = pd.read_csv('../agg_datasets/TouchEvent.csv')
keyPress = pd.read_csv('./../Our Data/KeyboardTouchEvent.csv',index_col = 0)

In [50]:
#keyPress['SessionID'] = np.floor(keyPress.ActivityID/10e6)
keyPress['SessionID'] = keyPress.ActivityID

In [51]:
skeleton = pd.read_csv('skeleton.csv')

In [52]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

In [53]:
#keyPress.to_csv('../windowed/TouchEvent.csv',index=False)
keyPress.to_csv('../Our Data Windowed/TouchEvent.csv',index=False)

# Accelerometer

In [54]:
keyPress = pd.read_csv('./../Our Data/Accelerometer.csv',index_col = 0)

In [55]:
keyPress['SessionID'] = keyPress.ActivityID

In [56]:
skeleton = pd.read_csv('skeleton.csv')

In [57]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

100000
200000
300000
400000


In [58]:
keyPress.to_csv('../Our Data Windowed/Accelerometer.csv',index=False)

# Gyroscope

In [59]:
keyPress = pd.read_csv('./../Our Data/Gyroscope.csv',index_col = 0)

In [60]:
keyPress['SessionID'] = keyPress.ActivityID

In [61]:
skeleton = pd.read_csv('skeleton.csv')

In [62]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

In [63]:
keyPress.to_csv('../Our Data Windowed/Gyroscope.csv',index=False)

# Magnetometer

In [64]:
keyPress = pd.read_csv('./../Our Data/Magnetometer.csv',index_col = 0)

In [65]:
keyPress['SessionID'] = keyPress.ActivityID

In [66]:
skeleton = pd.read_csv('skeleton.csv')

In [67]:
aux_list = []
last_session = -1
last_windowNumber = -1
last_windowEnd = -1
count = 0
j = 0
for _,row in keyPress.iterrows():
    if row.SessionID != last_session:
        skeletonWithSessionId = skeleton[skeleton.SessionID == row.SessionID]
        last_session = row.SessionID
    currentSystime = int(row.Systime)
    if currentSystime > last_windowEnd:
        final_df = skeletonWithSessionId[(skeletonWithSessionId.WindowStart <= currentSystime) & (skeletonWithSessionId.WindowEnd > currentSystime)]
        if final_df.empty:
            last_windowEnd = -1
            last_windowNumber = -1
        else:
            last_windowEnd = final_df.WindowEnd.values[0]
            last_windowNumber = final_df.WindowNumber.values[0]
    aux_list.append(last_windowNumber)
    count = count + 1
    j = j + 1
    if (j == 100000):
        print(count)
        j = 0
keyPress['WindowNumber'] = aux_list

In [68]:
keyPress.to_csv('../Our Data Windowed/Magnetometer.csv',index=False)